##### Copyright 2021 Habana Labs, Ltd. an Intel Company.

# Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and limitations under the License.

# DistilBert Sequence Classification with IMDb Reviews

**An adaptation of [Huggingface Sequence Classification with IMDB Reviews](https://github.com/huggingface/notebooks/blob/master/transformers_doc/pytorch/custom_datasets.ipynb) using Habana Gaudi AI processors.**

## Overview

This tutorial will take you through one example of using Huggingface Transformers models with IMDB datasets. The guide shows the workflow for training the model using Gaudi and is meant to be illustrative rather than definitive. 

Note: The dataset can be explored in the Huggingface model hub (IMDb), and can be alternatively downloaded with the Huggingface NLP library with load_dataset("imdb").

## Setup

Let’s start by downloading the dataset from the Large Movie Review Dataset webpage.

In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2022-11-17 06:50:07--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  47.8MB/s    in 1.7s    

2022-11-17 06:50:08 (47.8 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [2]:
!tar -xf aclImdb_v1.tar.gz

### Install required libraries
We will install the Habana version of transformers inside the docker.

In [3]:
!git clone --depth=1 https://github.com/HabanaAI/Model-References.git

Cloning into 'Model-References'...
remote: Enumerating objects: 6099, done.
remote: Counting objects: 100% (6099/6099), done.
remote: Compressing objects: 100% (4350/4350), done.
remote: Total 6099 (delta 1809), reused 5230 (delta 1582), pack-reused 0
Receiving objects: 100% (6099/6099), 39.51 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (1809/1809), done.


In [4]:
pip install Model-References/PyTorch/nlp/finetuning/huggingface/bert/transformers/.

Processing ./Model-References/PyTorch/nlp/finetuning/huggingface/bert/transformers
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 113.5 MB/s eta 0:00:00 0:00:01
  Created wheel for transformers: filename=transformers-4.20.1-py3-none-any.whl size=4248789 sha256=1a9f15e173edb71e1a91a02dad9735f6c2bb2079087b9f4424b9dc6a8e57adf1
  Stored in directory: /tmp/pip-ephem-wheel-cache-olf_mzup/wheels/99/23/5e/6fb06b86f9a4787183569a5ba0fe05827108d9cc054a944f9a
Successfully built transformers

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 279.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 333.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 361.0 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 11.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 338.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 314.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 221.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 319.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


This data is organized into pos and neg folders with one text file per example.

In [7]:
import os
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('./aclImdb/train')
test_texts, test_labels = read_imdb_split('./aclImdb/test')

We now have a train and test dataset, but let’s also also create a validation set which we can use for for evaluation and tuning without training our test set results. Sklearn has a convenient utility for creating such splits:

In [8]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

Alright, we’ve read in our dataset. Now let’s tackle tokenization. We’ll eventually train a classifier using pre-trained DistilBert, so let’s use the DistilBert tokenizer.

In [9]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28.0/28.0 [00:00<00:00, 47.6kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 226k/226k [00:00<00:00, 770kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455k/455k [00:00<00:00, 1.90MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 483/483 [00:00<00:00, 398kB/s]
The tokenizer class you load from this checkpoin

Now we can simply pass our texts to the tokenizer. We’ll pass truncation=True and padding=True, which will ensure that all of our sequences are padded to the same length and are truncated to be no longer than the model’s maximum input length. This will allow us to feed batches of sequences into the model at the same time.

In [10]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

## Building the model and Fine-tuning with Trainer on Gaudi

Now, let’s turn our labels and encodings into a Dataset object. In PyTorch, this is done by subclassing a torch.utils.data.Dataset object and implementing __len__ and __getitem__. We put the data in this format so that the data can be easily batched such that each key in the batch encoding corresponds to a named parameter of the forward() method of the model we will train.

In [11]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

The steps above prepared the datasets in the way that the trainer is expected. Now all we need to do is create a model to fine-tune, define the TrainingArguments and instantiate a Trainer. Next, let's enable the training on Gaudi by setting the variables in TrainingArguments. 
- The argument use_hpu is to set default device being Gaudi;
- The argument hmp is to enable mixed precision; 
- ops_hmp_bf16 and ops_hmp_fp32 files are required to specify the BF16 op list and BF16 op list; 
- The hmp_verbose controls the printout of datatype conversion between BF16 and FP32. 
In this example, we set hmp_verbose=False for a clean output.

In [12]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    use_hpu=True,
    use_lazy_mode=True,
    use_fused_adam=True,
    use_fused_clip_norm=True,
    hmp=True,
    hmp_bf16='./ops_bf16_distilbert_pt.txt',
    hmp_fp32='./ops_fp32_distilbert_pt.txt',
    hmp_verbose=False,
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

/usr/local/lib/python3.8/dist-packages/habana_frameworks/torch/utils/distributed_utils.py:6: UserWarning: habana_frameworks.torch.utils.distributed_utils.initialize_distributed_hpu is deprecated. Please use habana_frameworks.torch.distributed.hccl.initialize_distributed_hpu
  warnings.warn("habana_frameworks.torch.utils.distributed_utils.initialize_distributed_hpu is deprecated. "
--------------------------------------------------------------------------
An invalid value was supplied for an enum variable.

  Variable     : btl_vader_single_copy_mechanism
  Value        : non
  Valid values : 1:"cma", 4:"emulated", 3:"none"
--------------------------------------------------------------------------


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Now we can train the model from the previously saved checkpoint or the pretrained model. The default set of the full training is 3 epochs.

In [14]:
if not os.path.isdir("./results/checkpoint-3500"):
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
    trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
    )
    trainer.train()
else:
    model = DistilBertForSequenceClassification.from_pretrained("./results/checkpoint-3500")

loading configuration file ./results/checkpoint-3500/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "vocab_size": 30522
}

loading weights file ./results/checkpoint-3500/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at ./results/checkpoint-3500.
I

After the training finishes, we can evaluate the training results using the validation dataset. The function compute_metrics is used to calculate the accuracy number.

In [15]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

/tmp/ipykernel_120/3142568293.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
Enabled lazy mode


hmp:verbose_mode  False
hmp:opt_level O1


## Print out the final results

At the end of the training, we can print out the final training/evaluation result.

In [16]:
print("**************** Evaluation below************")
metrics = trainer.evaluate()
metrics["eval_samples"] = len(val_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64


**************** Evaluation below************


***** eval metrics *****
  eval_accuracy           =     0.9266
  eval_loss               =     0.3263
  eval_runtime            = 0:00:21.82
  eval_samples            =       5000
  eval_samples_per_second =    229.112
  eval_steps_per_second   =       3.62


## Gaudi training tips based trainer in huggingface transformers

In TrainingArguments setup:
- Set use_hpu=True to enable Gaudi device.
- Set use_lazy_mode=True to enable lazy mode for better performance.
- Set use_fused_adam=True to use Gaudi customized adam optimizer for better performance.
- Set use_fused_clip_norm=True to use Gaudi customized clip_norm kernel for better performance.
- Set mixed precision hmp=True.
    * The default hmp_verbose value is True. The setting hmp_verbose=False helps a clean printout.
    * For mixed precision, the following flags are needed.
      * hmp_bf16='./ops_bf16_distilbert_pt.txt',
      * hmp_fp32='./ops_fp32_distilbert_pt.txt',


## Summary

One can easily enable their model script on Gaudi by specifying a few Gaudi arguments in TrainingArguments.